In [28]:
import pandas as pd
import time
import numpy as np
import re
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import WordNetLemmatizer
import os
import emoji

In [2]:
for dirname, _, filenames in os.walk('../../data'):
    for filename in filenames:
        print(os.path.join(dirname, filename).replace('\\','/'))

../../data/clean-twitter_en.csv
../../data/clean_public.csv
../../data/cyberbullying_tweets.csv
../../data/labeled_tweets.csv
../../data/public_data_labeled.csv
../../data/twitter_en.csv


In [12]:
df = pd.read_csv('../../data/cyberbullying_tweets.csv')
df.head()

,tweet_text,cyberbullying_type
0,"In other words #katandandre, your food was cra...",not_cyberbullying
1,Why is #aussietv so white? #MKR #theblock #ImA...,not_cyberbullying
2,@XochitlSuckkks a classy whore? Or more red ve...,not_cyberbullying
3,"@Jason_Gio meh. :P thanks for the heads up, b...",not_cyberbullying
4,@RudhoeEnglish This is an ISIS account pretend...,not_cyberbullying


In [15]:
df.loc[:,'bulley_type'] = df.cyberbullying_type.apply(lambda x:0 if x=='not_cyberbullying' else 1)
df['bulley_type'].value_counts()

1    39747
0     7945
Name: bulley_type, dtype: int64

In [29]:
#remove contractions
def decontract(text):
    text = re.sub(r"can\'t", "can not", text)
    text = re.sub(r"n\'t", " not", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"\'s", " is", text)
    text = re.sub(r"\'d", " would", text)
    text = re.sub(r"\'ll", " will", text)
    text = re.sub(r"\'t", " not", text)
    text = re.sub(r"\'ve", " have", text)
    text = re.sub(r"\'m", " am", text)
    return text
def strip_emoji(text):
    return re.sub(emoji.get_emoji_regexp(), r"", text) #remove emoji
#1.build lemmatize models, and stopwords dictionary
stopwords = stopwords.words('English')
# the pipeline to process the raw text data
def pipeline(text:str):
    lm = WordNetLemmatizer()
    input = text.lower()
    input=re.sub("\\W"," ",input) # remove special chars
    str_input = re.sub(pattern=r'[^a-zA-z]',repl=' ',string=input) #remove words except digit and words
#     str_output = correct_sentence_spelling(sentence = str_input) # it is too time-consuming
    token_list = word_tokenize(str_input)
    token_list = [lm.lemmatize(i) for i in token_list]
    word_list = [word for word in token_list if word not in stopwords]
    word_str = ' '.join(word_list)
    return word_str

In [25]:
def preprocess(text):
    # text = strip_emoji(text)
    text = decontract(text)
    text = pipeline(text)
    return text

In [22]:
df.rename(columns={'tweet_text':'text'},inplace=True)
df.head()

,text,cyberbullying_type,bulley_type
0,"In other words #katandandre, your food was cra...",not_cyberbullying,0
1,Why is #aussietv so white? #MKR #theblock #ImA...,not_cyberbullying,0
2,@XochitlSuckkks a classy whore? Or more red ve...,not_cyberbullying,0
3,"@Jason_Gio meh. :P thanks for the heads up, b...",not_cyberbullying,0
4,@RudhoeEnglish This is an ISIS account pretend...,not_cyberbullying,0


In [33]:
texts_cleaned = []
for t in df.text:
    texts_cleaned.append(preprocess(t))
df.loc[:,'texts_cleaned'] = np.array(texts_cleaned)
df.head()

,text,cyberbullying_type,bulley_type,texts_cleaned
0,"In other words #katandandre, your food was cra...",not_cyberbullying,0,word katandandre food wa crapilicious mkr
1,Why is #aussietv so white? #MKR #theblock #ImA...,not_cyberbullying,0,aussietv white mkr theblock imacelebrityau tod...
2,@XochitlSuckkks a classy whore? Or more red ve...,not_cyberbullying,0,xochitlsuckkks classy whore red velvet cupcake
3,"@Jason_Gio meh. :P thanks for the heads up, b...",not_cyberbullying,0,jason_gio meh p thanks head concerned another ...
4,@RudhoeEnglish This is an ISIS account pretend...,not_cyberbullying,0,rudhoeenglish isi account pretending kurdish a...


In [34]:
df.to_csv('../../data/clean_kaggle.csv')